## Preprocessing

#### Using Buckwalter translitration
- o سكون
- a فتحة
- i كسرة
- ~ شدة
- u ضمة
- N تنوين بالضم
- F تنوين بالفتح
- K تنوين بالكسر

In [2]:
# Imports
import re
from lang_trans.arabic import buckwalter

### Constants

In [13]:
DIACRITICS = ['o', 'a', 'i', '~', 'u', 'N', 'F', 'K', ' ']

### Word Level Preprocessing

In [3]:
def remove_undesired_characters(sentence):
    # Except: . ? ! ,
    pattern = re.compile('[\[\]\\/@#\$&%\^\+<=>(){}\*\|\`:;\'"\~_!\.\?؟\,،؛-]')
    return re.sub(pattern, '', sentence)

def remove_html_tags(sentence):
    pattern = re.compile('<.*?>')
    return re.sub(pattern, '', sentence)

def remove_numbers(sentence):
    pattern = re.compile('[٠-٩0-9]')
    return re.sub(pattern, '', sentence)

def remove_english_letters(sentence):
    pattern = re.compile('[a-zA-Z]')
    return re.sub(pattern, '', sentence)

def data_cleaning(sentence):
    sentence = remove_english_letters(sentence)
    sentence = remove_numbers(sentence)
    sentence = remove_html_tags(sentence)
    sentence = remove_undesired_characters(sentence)
    
    # Remove extra spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    sentence = ' '.join(sentence.split())
    return sentence

def tokenization(sentence):
    # tokenizer = Tokenizer(num_words=None,
    # filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
    # lower = False, split = ' ')
    return sentence.split()

def word_level_preprocess(sentence):
    cleanedSentence = data_cleaning(sentence)
    finalSentence = tokenization(cleanedSentence)
    return finalSentence

### Buckwalter

In [7]:
def run_buckwalter(sentence):
    sentence = word_level_preprocess(sentence)
    output = []
    for word in sentence:
        output.append(buckwalter.transliterate(word))
    return output

def to_arabic(sentence):
    output = []
    for word in sentence:
        output.append(buckwalter.untransliterate(word))
    return output

### Remove Diacritics

In [47]:
# Function used to divide each word to the following tuple
# (each char of the word, its corresponding diacritic)
def extract_labels(sentences):
    output = []
    for word in sentences:
        diacritics = []
        word_char = []
        skip = False


        for i in range(len(word)):
            if not skip:
                if word[i] not in DIACRITICS:
                    word_char.append(word[i])
                    if i + 1 < len(word) and word[i + 1] not in DIACRITICS:
                        diacritics.append(" ")
                else:
                    if word[i] == "~":
                        if i + 1 < len(word) and word[i + 1] in DIACRITICS:
                            diacritics.append(f"~{word[i + 1]}")
                            skip = True
                    else:
                        diacritics.append(word[i])
            else: skip = False
        
        output.append((word_char, diacritics))

    return output

### Main

In [46]:

# sentence = "( قَوْلُهُ لِعَدَمِ مَا تَتَعَلَّقُ إلَخْ ) أَيْ الْوَصِيَّةُ ( قَوْلُهُ مَا مَرَّ ) أَيْ قُبَيْلَ قَوْلِ الْمَتْنِ لَغَتْ وَلَوْ اقْتَصَرَ عَلَى أَوْصَيْت لَهُ بِشَاةٍ أَوْ أَعْطُوهُ شَاةً وَلَا غَنَمَ لَهُ عِنْدَ الْمَوْتِ هَلْ تَبْطُلُ الْوَصِيَّةُ أَوْ يُشْتَرَى لَهُ شَاةٌ وَيُؤْخَذُ مِنْ قَوْلِهِ الْآتِي كَمَا لَوْ لَمْ يَقُلْ مِنْ مَالِي وَلَا مِنْ غَنَمِي أَنَّهَا لَا تَبْطُلُ ، وَعِبَارَةُ الْكَنْزِ وَلَوْ لَمْ يَقُلْ مِنْ مَالِي وَلَا مِنْ غَنَمِي لَمْ يَتَعَيَّنْ غَنَمُهُ إنْ كَانَتْ انْتَهَتْ ا ه سم ( قَوْلُهُ فَيُعْطَى وَاحِدَةً مِنْهَا إلَخْ ) كَمَا لَوْ كَانَتْ مَوْجُودَةً عِنْدَ الْوَصِيَّةِ وَالْمَوْتِ ، وَلَا يَجُوزُ أَنْ يُعْطَى وَاحِدَةً مِنْ غَيْرِ غَنَمِهِ فِي الصُّورَتَيْنِ وَإِنْ تَرَاضَيَا ؛ لِأَنَّهُ صُلْحٌ عَلَى مَجْهُولٍ مُغْنِي وَنِهَايَةٌ قَالَ ع ش قَوْلُهُ وَاحِدَةً مِنْهَا أَيْ كَامِلَةً ، وَلَا يَجُوزُ أَنْ يُعْطَى نِصْفَيْنِ مِنْ شَاتَيْنِ ؛ لِأَنَّهُ لَا يُسَمَّى شَاةً وَقَوْلُهُ وَلَا يَجُوزُ أَنْ يُعْطَى وَاحِدَةً مِنْ غَيْرِ غَنَمِهِ وَيَنْبَغِي أَنْ يُقَالَ مِثْلُ ذَلِكَ فِي الْأَرِقَّاءِ ا ه ."
sentence = "وَقِيلَ الْإِمْلَاءُ بِلَا إشْهَادٍ لَا يَكُونُ حُجَّةً وَالْأَوَّلُ أَظْهَرُ وَغَيْرُ مُسْتَبِينٍ كَالْكِتَابَةِ عَلَى الْهَوَاءِ أَوْ الْمَاءِ وَهُوَ بِمَنْزِلَةِ كَلَامٍ غَيْرِ مَسْمُوعٍ وَلَا يَثْبُتُ بِهِ شَيْءٌ مِنْ الْأَحْكَامِ وَإِنْ نَوَى ا ه وَمِثْلُهُ فِي الْهِدَايَةِ وَفَتَاوَى قَاضِي خَانْ وَحَاصِلُهُ أَنَّ الْأَوَّلَ صَرِيحٌ وَالثَّانِيَ كِنَايَةٌ وَالثَّالِثَ لَغْوٌ وَسُئِلَ قَارِئُ الْهِدَايَةِ عَنْ شَخْصٍ ادَّعَى عَلَى شَخْصٍ بِحَقٍّ وَأَظْهَرَ خَطَّ يَدِهِ بِذَلِكَ وَأَنْكَرَ الْمُدَّعَى عَلَيْهِ هَلْ يُحَلِّفُهُ الْقَاضِي أَنَّهَا لَيْسَتْ خَطَّهُ أَمْ عَلَى عَدَمِ الِاسْتِحْقَاقِ أَوْ يَسْتَكْتِبُهُ فَأَجَابَ إذَا كَتَبَ عَلَى رَسْمِ الصُّكُوكِ وَجَحَدَ أَنَّهُ خَطُّهُ يَحْلِفُ عَلَى أَنَّهُ لَيْسَ بِخَطِّهِ ؛ لِأَنَّهُ أَنْكَرَ الْكِتَابَةَ أَوْ يَسْتَكْتِبُهُ الْقَاضِي فَإِذَا كَتَبَ وَقَالَ أَهْلُ الْخِبْرَةِ هُمَا وَاحِدٌ أَلْزَمَهُ الْحَقَّ وَإِنْ اعْتَرَفَ أَنَّهُ خَطُّهُ وَأَنْكَرَ مَا كَتَبَ فِيهِ حَلَفَ الْمُقَرُّ لَهُ أَنَّ الْمُقَرَّ بِهِ قَبَضَهُ وَقَضَى لَهُ وَإِنْ لَمْ يَحْلِفْ لَا يَقْضِي لَهُ وَأَجَابَ عَنْ سُؤَالٍ آخَرَ إذَا كَتَبَ عَلَى رَسْمِ الصُّكُوكِ يَلْزَمُهُ الْمَالُ وَهُوَ أَنْ يَكْتُبَ يَقُولُ فُلَانُ بْنُ فُلَانٍ الْفُلَانِيُّ إنَّ فِي ذِمَّتِهِ لِفُلَانِ بْنِ فُلَانٍ الْفُلَانِيِّ كَذَا كَذَا فَهُوَ إقْرَارٌ( 4 / 427 )"
after_buckwalter = run_buckwalter(sentence)

# print("After Buckwalter:", after_buckwalter)
# print("After converting to arabic:", to_arabic(after_buckwalter))

print(extract_labels(after_buckwalter))

[(['w', 'q', 'y', 'l'], ['a', 'i', ' ', 'a']), (['A', 'l', '<', 'm', 'l', 'A', "'"], [' ', 'o', 'i', 'o', 'a', ' ', 'u']), (['b', 'l', 'A'], ['i', 'a']), (['<', '$', 'h', 'A', 'd'], [' ', 'o', 'a', ' ', 'K']), (['l', 'A'], ['a']), (['y', 'k', 'w', 'n'], ['a', 'u', ' ', 'u']), (['H', 'j', 'p'], ['u', '~a', 'F']), (['w', 'A', 'l', '>', 'w', 'l'], ['a', ' ', 'o', 'a', '~a', 'u']), (['>', 'Z', 'h', 'r'], ['a', 'o', 'a', 'u']), (['w', 'g', 'y', 'r'], ['a', 'a', 'o', 'u']), (['m', 's', 't', 'b', 'y', 'n'], ['u', 'o', 'a', 'i', ' ', 'K']), (['k', 'A', 'l', 'k', 't', 'A', 'b', 'p'], ['a', ' ', 'o', 'i', 'a', ' ', 'a', 'i']), (['E', 'l', 'Y'], ['a', 'a']), (['A', 'l', 'h', 'w', 'A', "'"], [' ', 'o', 'a', 'a', ' ', 'i']), (['>', 'w'], ['a', 'o']), (['A', 'l', 'm', 'A', "'"], [' ', 'o', 'a', ' ', 'i']), (['w', 'h', 'w'], ['a', 'u', 'a']), (['b', 'm', 'n', 'z', 'l', 'p'], ['i', 'a', 'o', 'i', 'a', 'i']), (['k', 'l', 'A', 'm'], ['a', 'a', ' ', 'K']), (['g', 'y', 'r'], ['a', 'o', 'i']), (['m', 's', 